In [3]:
#### IMPORT LIBRARIES

import numpy as np
# import all skimage related stuff
import skimage.io
from skimage.measure import label
from skimage import filters, exposure, restoration
from skimage import morphology

# import all napari related stuff
import napari
from napari.types import ImageData, LabelsData, LayerDataTuple
from napari.layers import Image, Layer, Labels
from magicgui import magicgui

# # import UI for stack selection
import tkinter as tk
from tkinter import filedialog

%gui qt5
import os

#### PROCESSING FUNCTIONS

def adaptive_local_threshold(image, block_size):
    # adaptive_local_threshold is a function that takes in an image and applies an odd-integer block size
    # kernel (or filter) and thresholds based on local spatial information. 
    
    return filters.threshold_local(image, block_size)

def gamma_level(image, gamma):
    # gamma_level is a function that takes in an image and changes the contrast by scaling the image 
    # by a factor "gamma".
    
    return exposure.adjust_gamma(image, gamma)

def global_threshold_method(image, Threshold_Method):
    # global_threshold_method is a function that allows a user to choose what kind of method to binarize
    # an image to create a mask. For a given method, a threshold will be calculated and returns a binarized 
    # image. 
    
    if Threshold_Method == 'Isodata':
        thresh = filters.threshold_isodata(image) # calculate threshold using isodata method
        tmp_img = image > thresh
        return binary_labels(tmp_img)
    if Threshold_Method == 'Li':
        thresh = filters.threshold_li(image) # calculate threshold using isodata method
        tmp_img = image > thresh
        return binary_labels(tmp_img)
    if Threshold_Method == 'Mean':
        thresh = filters.threshold_mean(image) # calculate threshold using isodata method
        tmp_img = image > thresh
        return binary_labels(tmp_img)
    if Threshold_Method == 'Minimum':
        thresh = filters.threshold_minimum(image)
        tmp_img = image > thresh
        return binary_labels(tmp_img)
    if Threshold_Method == 'Otsu':
        thresh = filters.threshold_otsu(image)
        tmp_img = image > thresh
        return binary_labels(tmp_img)
    if Threshold_Method == 'Yen':
        thresh = filters.threshold_yen(image)
        tmp_img = image > thresh
        return binary_labels(tmp_img)
    if Threshold_Method == 'Triangle':
        thresh = filters.threshold_triangle(image)
        tmp_img = image > thresh
        return binary_labels(tmp_img)
    
def binary_labels(image):
    # function binary_labels labels the entire neuron and entries of the Image = 2. Later, 2 => 'Dendrites'
    temp_img = image.copy()
    temp_img[temp_img > 0] = 2
    temp_img = temp_img.astype('int')
    return temp_img
    
#### MAIN WIDGET/PROGRAM HERE

@magicgui(
    gamma = {"widget_type": "FloatSlider", 'max': 2},
    block_size = {"widget_type": "SpinBox", 'label': 'Block Size:', 'min': 1, 'max': 20},
    threshold_method = {"choices": ['Isodata', 'Li', 'Mean', 'Minimum', 'Otsu', 'Triangle', 'Yen']},
    layout = 'vertical'
 )

def threshold_widget(image: ImageData, 
                     gamma = 1,
                     block_size = 3,
                     threshold_method = 'Yen',
                     ) -> LayerDataTuple:
    #function threshold_widget does a series of image processing and thresholding to binarize the image and make a label
    
    if image is not None:
        # adjust the gamma levelz
        label_img = gamma_level(image, gamma)
        
        # go through the stack and perform the local threshold
        for curr_stack in range(np.shape(label_img)[0]):
            label_img[curr_stack] = adaptive_local_threshold(label_img[curr_stack], block_size)
        
        # finally do a global threshold to calculate optimized value to make it black/white
        label_img = global_threshold_method(label_img, threshold_method)
        label_img = binary_labels(label_img)

        return (label_img, {'name': 'neuron_label'}, 'labels')

#### WIDGET FOR PROCESSING IMAGE AND SHOWING THE PROCESSED IMAGE BEFORE SEGMENTATION
# from magicgui import widgets

@magicgui(
    filter_method = {"choices": ['None','median', 'gaussian', 'bilateral', 'TV']},
    value_slider = {"widget_type": "FloatSlider", 'max': 4, 'label': 'None'},
    layout = 'vertical'
 )
def filter_widget(image: ImageData,
                  filter_method = 'None',
                  value_slider = 1) -> LayerDataTuple:
    # filter_widget is a function that takes an input image and selects a filter method 
    # for denoising an image. 
    # Returns an IMAGE layer.
    
    if image is not None:
        stack_size = np.shape(image)[0]
        if filter_method == 'median': # use a median filter and go through the entire stack to apply the filter
            tmp_img = image.copy()
            for curr_stack in range(stack_size):
                tmp_img[curr_stack] = filters.median(tmp_img[curr_stack], morphology.disk(value_slider))
            return (tmp_img, {'name': 'smoothened_image'}, 'image')
        
        if filter_method == 'gaussian': # use a gaussian filter 
            tmp_img = image.copy()
            tmp_img = filters.gaussian(tmp_img, sigma = value_slider)
            return (tmp_img, {'name': 'smoothened_image'}, 'image')
        
        if filter_method == 'bilateral': # use a bilateral filter 
            tmp_img = image.copy()
            for curr_stack in range(stack_size):
                tmp_img[curr_stack] = restoration.denoise_bilateral(tmp_img[curr_stack], sigma_spatial = value_slider)
            
            return (tmp_img, {'name': 'smoothened_image'}, 'image')
        
        if filter_method == 'TV': # using a total-variation (TV) denoising filter
            tmp_img = image.copy()
            for curr_stack in range(stack_size):
                tmp_img[curr_stack] = restoration.denoise_tv_chambolle(tmp_img[curr_stack], weight = value_slider)
            return (tmp_img, {'name': 'smoothened_image'}, 'image')
            
@filter_widget.filter_method.changed.connect
def change_label(event):
    # change_label function is written to change the label of the FloatSlider widget
    # such that the user won't be confused as to what metric is being used. 
    
    if event.value == 'median':
        filter_widget.value_slider.label = 'Pixel Radius'
    if event.value == 'gaussian':
        filter_widget.value_slider.label = 'sigma'
    if event.value == 'bilateral':
        filter_widget.value_slider.label = 'sigma_spatial'
    if event.value == 'TV':
        filter_widget.value_slider.label = 'weight'
        filter_widget.value_slider.max = 1
        filter_widget.value_slider.value = 0


#####################################################################################

#### WIDGET FOR SAVING LAYER AS H5 FILE

import h5py

@magicgui(
    call_button = 'Save Layer',
    file_picker = {"widget_type": 'FileEdit', 'value': 'N/A', 'mode': 'd'})
def save_layer(image: ImageData, label: Labels, file_picker = 'N/A'):
    folder_name = file_picker
    label_name = label.name # access label
    h5_name = label_name + '.h5'
    full_dir = os.path.join(folder_name, h5_name)
    
    # Dictionary for label ints 
    label_dict = {
            'Background' : 0,
            'Soma' : 1,
            'Dendrite' : 2,
            'Filopodia' : 3,
            'Axon' : 4,
            'Growth Cone' : 5,
            'Autofluorescence' : 6,
            'Melanocyte' : 7,
            'Noise' : 8,
        
    }
    # initialize HDF5 file
    hf = h5py.File(full_dir, 'w') # 'w' is for writing
    hf.create_dataset('raw_image', data = image)
    hf.create_dataset('label', data = label.data)
    hf.create_dataset('label_dict', data = label_dict)

    hf.close()
    

#####################################################################################


# file_path = os.path.join(neuron_dir,neuron_file)

root = tk.Tk()
root.withdraw()

file_path = filedialog.askopenfilename()
neuron_image = skimage.io.imread(file_path)



viewer = napari.Viewer()
viewer.add_image(neuron_image, name = 'Neuron')
viewer.window.add_dock_widget(filter_widget)
viewer.window.add_dock_widget(threshold_widget)
viewer.window.add_dock_widget(save_layer)
napari.run()

# threshold_widget()